<a href="https://colab.research.google.com/github/MonikaVen/LLM-Prompting-RAG-Aligment-Training-Tutorial/blob/main/1-Mistral-7B-Effective-Prompting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Effective Prompting With Mistral 7B
Try different metatags.
Try Chain-of-Thought prompting



In [2]:
# You only need to run this once per machine
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

##Load and Test Mistral 7B

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print('Selected device:', device)
#device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")

prompt = "My favourite condiment is"

model_inputs = tokenizer([prompt], return_tensors="pt").to(device)
model.to(device)

generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=True, )
tokenizer.batch_decode(generated_ids)[0]


Selected device: cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

#Try different metatags for Mistral 7B

In [ ]:
def evaluate_prompt(prompt, tokenizer):
    with torch.no_grad():
        model_inputs = tokenizer([prompt], return_tensors="pt").to(device)
    generated_code = tokenizer.decode(model.generate(**model_inputs, max_new_tokens=256)[0], skip_special_tokens=True)
    return generated_code

eval_prompt = """Print hello world in python c and c++"""
tokenizer.pad_token = None
result = evaluate_prompt(eval_prompt, tokenizer)
print(f"Using token: {tokenizer.pad_token}, prompt result: ", result )
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
result = evaluate_prompt(eval_prompt, tokenizer)
print(f"Using token: {tokenizer.pad_token}, prompt result: ", result )
tokenizer.pad_token = tokenizer.eos_token
result = evaluate_prompt(eval_prompt, tokenizer)
print(f"Using token: {tokenizer.pad_token}, prompt result: ", result )

## Load Mistral 7B Instruct and test Chain-of-Thought Prompting

In [ ]:
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")


messages_1 = [
    {"role": "user", "content": "The Cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they still have?"},
]

model_inputs = tokenizer.apply_chat_template(messages_1, return_tensors="pt").to("cuda")

generated_ids = model.generate(model_inputs, max_new_tokens=100, do_sample=True)
tokenizer.batch_decode(generated_ids)[0]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/utils/modeling.py:1363: UserWarning: Current model requires 603984384 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
messages_2 = [
    {"role": "user", "content": "The Cafeteria had 23 apples."},
    {"role": "assistant", "content": "Understood."},
    {"role": "user", "content": "They used 20 to make lunch."},
    {"role": "assistant", "content": "Alright."},
    {"role": "user", "content":  "Then they bought 6 more. How many apples do they still have?"}
]

model_inputs = tokenizer.apply_chat_template(messages_1, return_tensors="pt").to("cuda")

generated_ids = model.generate(model_inputs, max_new_tokens=100, do_sample=True)
tokenizer.batch_decode(generated_ids)[0]